<a href="https://colab.research.google.com/github/lsjlsj5846/error-unifiedQA/blob/main/FinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CommonsenseQA의 오답 유형 분석: Unified-QA 모델을 중심으로
인문대학 언어학과 | 2018-18450 이소진

In [1]:
!pip install transformers
!pip install ratsnlp
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 34.8 MB/s 
     |████████████████████████████████| 182 kB 77.7 MB/s 
     |████████████████████████████████| 7.6 MB 62.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 681 kB/s 
     |████████████████████████████████| 2.8 MB 48.0 MB/s 
     |████████████████████████████████| 582 kB 69.5 MB/s 
     |████████████████████████████████| 57 kB 6.3 MB/s 
     |████████████████████████████████| 512 kB 79.0 MB/s 
     |████████████████████████████████| 3.3 MB 70.4 MB/s 
     |████████████████████████████████| 880 kB 55.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=0cfbc91b2fdf44bec73b3b304b66dcb22f44f3ca269b4cb934fbc8dbafabfb09
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf65

In [2]:
!git clone https://github.com/lsjlsj5846/unifiedqa.git

Cloning into 'unifiedqa'...
remote: Enumerating objects: 353, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 353 (delta 57), reused 32 (delta 32), pack-reused 280
Receiving objects: 100% (353/353), 28.88 MiB | 16.92 MiB/s, done.
Resolving deltas: 100% (167/167), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")
  print("Running on the CPU")

In [6]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "allenai/unifiedqa-t5-small" # you can specify the model size here
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

def run_model(input_string, **generator_args):
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = model.generate(input_ids, **generator_args)
    return tokenizer.batch_decode(res, skip_special_tokens=True)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [12]:
import random
import jsonlines as js

json_file_path = "/content/drive/MyDrive/data/test_rand_split_no_answers.jsonl"

question_list = []

with js.open(json_file_path) as f:
  for line in f:
    question_list.append(line['question'])

sample_questions = []

for i in range(50):
  index = random.randint(0, 1139) # Number of total questions in dataset is 1140
  result = []

  result.append(question_list[index]['stem'])
  result.append('\n')
  
  choice = question_list[index]['choices']
  for j in range(5):
    result.append('(')
    result.append(choice[j]['label'].lower())
    result.append(')')
    result.append(' ')
    result.append(choice[j]['text'])
    result.append(' ')
  
  result_str = ''.join(result)
  result_str = result_str.strip()

  sample_questions.append(result_str)

In [17]:
count = 0
for question in sample_questions:
  count += 1
  print("Q" + str(count))
  print(question)
  print("result: " + str(run_model(question)))
  print()

Q1
The man owned a large property with a ranch house, but there wasn't much landscaping besides spiky stuff because it was located where?
(a) desert (b) countryside (c) texas (d) mountain range (e) montana
result: ['countryside']

Q2
James thought that killing people was dangerous.  He didn't want to waste his life by doing what?
(a) going to jail (b) get arrested (c) going to prison (d) murder (e) die
result: ['murder']

Q3
A place where a lot of people live is called what?
(a) classroom (b) hotel (c) populated areas (d) supermarket (e) race track
result: ['populated areas']

Q4
What do you need to do before you eat vegetables?
(a) clean (b) get gas (c) stand (d) prepare (e) open mouth
result: ['prepare']

Q5
What could a condominium be?
(a) florida (b) towels (c) pirates (d) michigan (e) complex
result: ['complex']

Q6
If a person works hard and applies themselves, what is the end result for them?
(a) will succeed (b) thank god (c) death (d) own property (e) bring home
result: ['own 